In [ ]:
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>")) 

In [ ]:
import json
import pprint 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import os
import boto3
import time
from pprint import pprint

# boto3を使って動画をrekognitionで分析する

In [ ]:
bucket = ""
prefix = ""
video_filename = ""

In [ ]:
rek_client = boto3.client("rekognition")

In [ ]:
def start_label_detection(client, bucket, prefix, name, **kwargs):
    response = rek_client.start_label_detection(
        Video={
            'S3Object':{
                'Bucket': bucket,
                'Name': os.path.join(prefix, name)
            }
        },
        **kwargs
    )
    return response



In [ ]:
start_response = start_label_detection(rek_client, bucket, prefix, video_filename)
print(start_response)

In [ ]:
start_response['JobId']

In [ ]:
def get_label_detection(client, job_id, **kwargs):
    get_response = client.get_label_detection(
        JobId=job_id,
        **kwargs)
    return get_response

In [ ]:
next_token=''
get_response_list=[]
while True:
    get_response = get_label_detection(rek_client, start_response['JobId'], NextToken=next_token)
    print("Call API (get_response)")
    get_response_list.append(get_response)
    if 'NextToken' in get_response.keys():
        next_token = get_response['NextToken']
        print(next_token)
    else:
        print("No next token, stop API call")
        break

In [ ]:
len(get_response_list)

In [ ]:
[g['ResponseMetadata'] for g in get_response_list]

# Rekognitionの結果の読み込みと整形

In [ ]:
get_response.keys()

In [ ]:
filename = "data/shibuya2.json"

In [ ]:
def parse_video_label_detection(json_filename, start_time=None):
    
    body, ext = os.path.splitext(json_filename)
    assert ext == '.json'
    
    # jsonファイルを開く
    with open(json_filename) as f:
        input_json = json.load(f)
    #物体検出の結果のみを抽出
    ext_obj = [f for f in input_json['Labels'] if not f['Label']['Instances'] == []]   # 物体検出の結果はInstancesに格納されている
    
    # 物体検出の結果だけを含むjson文字列（dict）を作成
    obj_json = input_json.copy()    # 大元のjsonファイルをコピー
    obj_json['Labels'] = ext_obj # 抽出した物体検出結果で置き換え
    
    out_json = "{}_detected_object.json".format(body)
    with open(out_json, 'w') as f:
        json.dump(obj_json, f)
    print('Create json file to "{}"'.format(out_json))
    
    # 物体検出結果をDataFrameに整形
    # ネストしているラベル (Label-Instances)をリスト形式でrecord_pathで指定
    obj_df = pd.json_normalize(obj_json['Labels'], record_path=['Label', 'Instances'], \
                          meta=['Timestamp', ['Label', 'Name'], ])
    obj_df = obj_df[['Timestamp', 'Label.Name', 'Confidence', 'BoundingBox.Width', 'BoundingBox.Height',
       'BoundingBox.Left', 'BoundingBox.Top',]]
    obj_df.columns= ['timedelta', 'name', 'confidence', 'width', 'height', 'left', 'top',]

    
    # 物体領域の中心点と面積を算出しDataFrameに追加
    obj_df['center_x'] = obj_df['left'] + obj_df['width']/2
    obj_df['center_y'] = obj_df['top'] - obj_df['height']/2
    obj_df['area'] = obj_df['width'] * obj_df['height']
    
    # jsonのタイムスタンプをTimeDeltra形式に変換
    timedelta = pd.to_timedelta(obj_df['timedelta'] * 10**6)
    
    # 開始時間を設定。starttime引数を指定しなかった場合は現在時間にする
    if start_time == None:
        start_time = datetime.datetime.now() #"2021-03-19 00:00:00"
    obj_df['timestamp'] = pd.to_datetime(start_time) + timedelta
    
    # 識別子（インデックス番号）入りのオブジェクト名を作成
    obj_df['name_with_id'] = obj_df['name'] + obj_df.index.map(lambda x: '_' + str(x))
    
    out_csv = "{}_detected_object.csv".format(body)
    obj_df.to_csv(out_csv, index=None)
    print('Create csv file to "{}"'.format(out_csv))
    
    return obj_df


In [ ]:
obj_df = parse_video_label_detection(filename)

In [ ]:
obj_df.tail(30)

# TimeStreamへのデータ登録

## データベース、テーブルの作成

In [ ]:
db_name = "sampleDB8"
table_name = "sampleTable8"

In [ ]:
ts_write = boto3.client("timestream-write")

データベース作成

In [ ]:
_response = ts_write.create_database(
    DatabaseName=db_name,
    Tags=[
    ]
)

In [ ]:
_response

テーブル作成

In [ ]:
_response = ts_write.create_table(
    DatabaseName=db_name,
    TableName=table_name,
    RetentionProperties={
        'MemoryStoreRetentionPeriodInHours': 123,
        'MagneticStoreRetentionPeriodInDays': 123
    },
    Tags=[
    ]
)
print(_response)

テーブルにレコードを登録

In [ ]:
dim_columns = ['name', 'name_with_id']
mes_columns = ['timedelta', 'confidence', 'width',  'height', 'left', 'top', 'center_x', 'center_y', 'area']


records = []
write_freq = 10

for index, row in obj_df.iterrows():
    print(index)
    
    dimensions = []
    for col in dim_columns:
        dimensions.append({
            'Name': col,
            'Value': str(row[col])
        })
    #print(dimensions)
    
    '''
    common_attributes = {
        'Dimensions': dimensions,
        'MeasureValueType': 'DOUBLE',
        'Time': str(round(time.time() * 1000))
        }
    '''
    
    for col in mes_columns:
        records.append({
            'Dimensions': dimensions,
            'MeasureName': col,
            'MeasureValue': str(row[col]),
            'MeasureValueType': 'DOUBLE',
            'Time': str(round(row['timestamp'].to_pydatetime().timestamp() * 10**6)),
            #'Time': str(round(time.time() * 10**6)),

            'TimeUnit': 'MICROSECONDS',

        })
    
    #print(records)
        
    if index % write_freq == 0:
        try:
            result = ts_write.write_records(DatabaseName=db_name, TableName=table_name,
                                               Records=records, CommonAttributes={})
            #print("WriteRecords Status: [%s]" % result['ResponseMetadata']['HTTPStatusCode'])
        except ts_write.exceptions.RejectedRecordsException as err:
            print("RejectedRecords: ", err)
            for rr in err.response["RejectedRecords"]:
                print("Rejected Index " + str(rr["RecordIndex"]) + ": " + rr["Reason"])
            print("Other records were written successfully. ")
        except Exception as err:
            print("Error:", err)
        records = []
        print("recoreds set to zero")

if index % write_freq != 0:
    try:
        result = ts_write.write_records(DatabaseName=db_name, TableName=table_name,
                                           Records=records, CommonAttributes={})
        #print("WriteRecords Status: [%s]" % result['ResponseMetadata']['HTTPStatusCode'])
    except ts_write.exceptions.RejectedRecordsException as err:
        print("RejectedRecords: ", err)
        for rr in err.response["RejectedRecords"]:
            print("Rejected Index " + str(rr["RecordIndex"]) + ": " + rr["Reason"])
        print("Other records were written successfully. ")
    except Exception as err:
        print("Error:", err)
    records = []
    print("recoreds set to zero")

In [ ]:
dimensions

In [ ]:
CommonAttributes={
        'Dimensions': dimensions
        'MeasureName': 'string',
        'MeasureValue': 'string',
        'MeasureValueType': 'DOUBLE'|'BIGINT'|'VARCHAR'|'BOOLEAN',
        'Time': 'string',
        'TimeUnit': 'MILLISECONDS'|'SECONDS'|'MICROSECONDS'|'NANOSECONDS',
        'Version': 123
    },

In [ ]:
_response = ts_write.write_records(
    DatabaseName=db_name,
    TableName=table_name,
    CommonAttributes={
        'Dimensions': [
            {
                'Name': 'string',
                'Value': 'string',
                'DimensionValueType': 'VARCHAR'
            },
        ],
        'MeasureName': 'string',
        'MeasureValue': 'string',
        'MeasureValueType': 'DOUBLE'|'BIGINT'|'VARCHAR'|'BOOLEAN',
        'Time': 'string',
        'TimeUnit': 'MILLISECONDS'|'SECONDS'|'MICROSECONDS'|'NANOSECONDS',
        'Version': 123
    },
    Records=[
        {
            'Dimensions': [
                {
                    'Name': 'string',
                    'Value': 'string',
                    'DimensionValueType': 'VARCHAR'
                },
            ],
            'MeasureName': 'string',
            'MeasureValue': 'string',
            'MeasureValueType': 'DOUBLE'|'BIGINT'|'VARCHAR'|'BOOLEAN',
            'Time': 'string',
            'TimeUnit': 'MILLISECONDS'|'SECONDS'|'MICROSECONDS'|'NANOSECONDS',
            'Version': 123
        },
    ]
)

In [ ]:
!pip freeze | grep pandas

In [ ]:
!pip install awswrangler

In [ ]:
rejected_records = wr.timestream.write(
    df=obj_df,
    database=db_name,
    table= table_name,
    time_col="TimeStamp",
    #measure_col= ["Confidence", "Width", "Height", "Left", "Top", "Center_X", "Center_Y", "Area"],
    measure_col= "Confidence",

    dimensions_cols=["Name"],
)

print(f"Number of rejected records: {len(rejected_records)}")


In [ ]:
rejected_records